In [1]:
import torch
from adversarial_attack.fgsm import FGSM
from adversarial_attack.pgd import PGD
from adversarial_attack.bim import BIM
from adversarial_attack.utils import compute_accuracy, compute_confusion_matrix

#### Create the HT model and import pretrained weights

In [3]:
import pickle
from dnn_model.CNN_netlist_softma_save_resluts import Classifier_Netlist

with open('save/source_config.pkl', 'rb') as pickle_file:
    source_config_copy = pickle.load(pickle_file)

path = '/home/erastus/Desktop/Postdoc_projects/Adv-TruDetect/json_temp_file/word2vec_emb/CNN_model_pretrained.pth'
HTnn_net = Classifier_Netlist(group_id=str(2), base_path='json_temp_file', source_config=source_config_copy, pretrained=path)   

Chunking...0/3
Read 1M Feature Traces.
Chunking...1/3
Read 2M Feature Traces.
Chunking...2/3
Read 3M Feature Traces.
Chunking...Last
Read 4M Feature Traces.
Iterable Dataset Loaded...
===TEST===
Chunking...0/1
Chunking...Last
Read 1M Feature Traces.
Iterable Dataset Loaded...
Model loaded


#### Evaluate the HT model on val_data

In [4]:
HTnn_net.evaluate_no_save()

2024-07-16 04:51:42,433 KFD[2]_nnclassifier INFO: ====TEST====


33716it [01:08, 493.18it/s]                         

2024-07-16 04:52:50,805 KFD[2]_nnclassifier INFO: Test Loss: 0.280729, Acc: 0.998739
2024-07-16 04:52:50,806 KFD[2]_nnclassifier INFO: tensor([[1.0767e+06, 2.9300e+02],
        [1.0400e+03, 8.4400e+02]])
2024-07-16 04:52:50,811 KFD[2]_nnclassifier INFO: tensor([0.9997, 0.4480])
2024-07-16 04:52:50,811 KFD[2]_nnclassifier INFO: tensor([0.9990, 0.7423])


#### Sample a subset of HT PCPs from the validation dataset

In [6]:
clean_data, clean_labels = [], []

n_samples = 10000
for i, sample_batched in enumerate(HTnn_net.val_dataloader):
    clean_data.append(torch.unsqueeze(sample_batched[0], 1))
    clean_labels.append(sample_batched[1])
    if i == n_samples:
        break

clean_data = torch.stack(clean_data, dim=0).to(HTnn_net.device)
clean_data = clean_data.view(clean_data.size(0)*clean_data.size(1), clean_data.size(2), clean_data.size(3), clean_data.size(4))
clean_labels = torch.stack(clean_labels, dim=0).view(clean_data.size(0)*clean_data.size(1)).to(HTnn_net.device)
clean_labels = clean_labels.view(clean_data.size(0)*clean_data.size(1))

mask = clean_labels[:] == 1
indices = torch.nonzero(mask).squeeze()

clean_data = clean_data[indices]
clean_labels = clean_labels[indices]

#### Run the Adversarial Attack (FGSM)

In [7]:
adv_attack = FGSM(HTnn_net.model, eps=8/255)
adv_data = adv_attack(clean_data, clean_labels)

robust_acc = compute_accuracy(HTnn_net.model, adv_data, clean_labels, batch_size=32, device=HTnn_net.device)
print("Robust Accuracy", robust_acc) 

cm = compute_confusion_matrix(HTnn_net.model, clean_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("Before Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

cm = compute_confusion_matrix(HTnn_net.model, adv_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("After Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

Robust Accuracy 0.18740849194729137
Before Adv Attack -- TN 0 | FP 0 | FN 115 | TP 568
After Adv Attack -- TN 0 | FP 0 | FN 555 | TP 128


#### Run the Adversarial Attack (PGD)

In [8]:
adv_attack = PGD(HTnn_net.model, eps=16/255)
adv_data = adv_attack(clean_data, clean_labels)

robust_acc = compute_accuracy(HTnn_net.model, adv_data, clean_labels, batch_size=32, device=HTnn_net.device)
print("Robust Accuracy", robust_acc) 

cm = compute_confusion_matrix(HTnn_net.model, clean_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("Before Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

cm = compute_confusion_matrix(HTnn_net.model, adv_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("After Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

Robust Accuracy 0.1493411420204978
Before Adv Attack -- TN 0 | FP 0 | FN 115 | TP 568
After Adv Attack -- TN 0 | FP 0 | FN 581 | TP 102


#### Run the Adversarial Attack (BiM)

In [9]:
adv_attack = BIM(HTnn_net.model, eps=8/255)
adv_data = adv_attack(clean_data, clean_labels)

robust_acc = compute_accuracy(HTnn_net.model, adv_data, clean_labels, batch_size=32, device=HTnn_net.device)
print("Robust Accuracy", robust_acc) 

cm = compute_confusion_matrix(HTnn_net.model, clean_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("Before Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

cm = compute_confusion_matrix(HTnn_net.model, adv_data, clean_labels, HTnn_net.device)
tn, fp, fn, tp = cm.ravel()
print("After Adv Attack -- TN {} | FP {} | FN {} | TP {}".format(tn, fp, fn, tp))

Robust Accuracy 0.0
Before Adv Attack -- TN 0 | FP 0 | FN 115 | TP 568
After Adv Attack -- TN 0 | FP 0 | FN 683 | TP 0
